# Prediction of NMR, X-ray and Mössbauer experimental results for amorphous Li-Si alloys using a novel DFTB model

This notebook contains the code needed to reproduce the results of our [arXiv article](https://arxiv.org/abs/2305.11006) where we propose nearest-neighbors models to predict experimental results in Silicon anodes, which hold a great promise for next-generation Li-ion batteries. 

**Disclaimers**
1. This will not follow the same order as the main text since the Supporting Material will be in the position where it is needed and not at the end of the notebook.
2. Data is available [here](https://github.com/fernandezfran/unveiled_LiSi_amorphous_structures), but we are going to use more extensive data for better statistics that cannot be uploaded to GitHub due to their size. We will also use NMR, X-ray and Mössbauer experimental data [[1](https://pubs.acs.org/doi/10.1021/ja8086278), [2](https://pubs.acs.org/doi/10.1021/ja108085d), [3](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.3460), [4](https://iopscience.iop.org/article/10.1149/1.3073879/meta)].

## Data collection

The data was obtained using [the set B of parameters](https://github.com/alexispaz/DFTB_LiSi/tree/main/lisi) for DFTB LiSi in [DFTB+ software](www.dftbplus.org), managed with a [lithiation code](https://github.com/fernandezfran/lithiation) that follows the protocol below:
1. Start with an [amorphous structure of silicon](https://pubs.acs.org/doi/full/10.1021/acs.jpca.3c00075).
2. Add a Li atom at the center of the largest spherical void (using voronoi vertices),
3. increase the volume and scale the coordinates by a factor to obtain the experimental expansion of the system.
4. Then, perform a NPT molecular dynamics equilibration for 10ps usign the Berendsen Thermostat and Barostat.
5. Repeat 2-4 steps until the desired number of Li atoms is reached.

In the module upload_data.py we have different functions to load the necessary data for each experiment.

In [1]:
import upload_data

## Distribution Functions

The Radial Distribution Function (RDF) describes the probability of finding an atom in a shell at a distance _r_ from a reference atom.

In [2]:
# code here

The nature of the obtained distributions is typical for amorphous structures, having a definite first peak at short _r_ and decreasing following ones for increasing _r_. The decrease of the first peak and the shift of the one indicates that Si-Si bonds are being broken and isolated Si atoms appears.

These RDFs can be used to compute the Pair Distribution Funcition, _G(r)_

In [3]:
# code here

and compare with experiments

In [4]:
# code here

## $^7$Li NMR chemical spectra

This Nuclear Magnetic Resonce (NMR) signal is characterized by a Voigt peak (V), which corresponds to the combination of a Lorentzian that is intrinsic to the NMR phenomena and a Gaussian due to the detector. Each Li atom introduce a shift on this signal that depends on the electromagnetic shielding caused by the local structure. We propose a nearest-neighbor model to emulate and interpret the features
in the NMR spectra, where a peak is obtained within the centers of each Li atom
$$
\delta_{\text{Li}} = \frac{1}{N_{\text{Si}}} \sum_{\text{Si} \in \textit{NN}} \delta_{\text{Key}},
$$
where the sum is over the Si nearest-neighbors (NN) and $\delta_{\text{Key}}$ can be 18 ppm if the Si neighbor is bonded and 6 ppm if it is isolated. Then the total spectra has an intensity
$$
I = \sum_{\text{Li} \in \text{Si}} V(\text{ppm}, \delta_{\text{Li}}, \sigma, \gamma),
$$
where $\sigma$ and $\gamma$ are the standard deviations and the half-width at half-maximum of gaussian and lorentzian components, respectively.

### Crystalline spectra

These atomic structures where obtained from [Materials Project](https://materialsproject.org/).

In [5]:
# code here

### Amorphous spectra

To facilitate the comparison with our model we includ a peak at -0.3 ppm, which represents the SEI contribution.

In [6]:
# code here

Discrepancies between model predictions and experiments stem from inhomogeneous lithiation that is not present in the simulated amorphous structures.

## Mössbauer spectra

This signal consist of two peaks that almost completely overlap at the extremes cases but are separeted for the intermediate cases. We reproduce the separation distance, $\Delta$, by tuning $a$ and $b$ parameters in
$$
\Delta = a \min\left\lbrace C_{\text{Li}}, C_{\text{Si}}\right\rbrace + b,
$$
where $C_{\text{Li}}$ and $C_{\text{Si}}$ are the concentration of Li and Si atoms.

In [7]:
# code here

An improvement is observed when considering the nearest-neighbors concentrations of the amorphous structures with respect to the overall concentrations.